### Setup very simple RAG (5 liner)

In [1]:
# Go one level up in the directories hierarchy to access src directory and codes
import sys
import os
# Add project root to Python path
project_root = os.path.abspath("..")  # go one level up from notebooks/
sys.path.append(project_root)

In [2]:
# Setup necessary models for chatting and embedding
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from core.config.config import Config
from google.genai import types

router_llm = GoogleGenAI(
    model = Config.CHAT_LLM,
    api_key = Config.GOOGLE_API_KEY,
    generation_config = types.GenerateContentConfig(
        thinking_config = types.ThinkingConfig(thinking_budget = 0),
        temperature = Config.LLM_TEMPERATURE,
    ),
    max_tokens = Config.LLM_MAX_TOKENS
)

embed_model = HuggingFaceEmbedding(
    model_name = Config.EMBEDDING_MODEL
)

c:\Users\Sanzhar\workspace\github.com\kaydarovv\ai_agent_learning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Setup simple RAG
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

docs_path = "../documents"

# 1) Read documents and create list of 'Document' objects, that has id_, metadata, text attributes.
#    Document class (generic container for any data source) is a subclass of the TextNode class 
documents = SimpleDirectoryReader(input_dir = docs_path).load_data()

# 2) Read each of this document objects and create index from it
#    Document objects are parsed into Node objects that have different attributes such as text, embeddings, metadata, relationships.
#    Document objects are split into multiple nodes (relationships between these nodes are recorded in Node objects as attributes).
index = VectorStoreIndex.from_documents(
    documents = documents,
    show_progress = True,
    embed_model = embed_model
)

Generating embeddings: 100%|██████████| 30/30 [00:01<00:00, 16.60it/s]


In [4]:
import nest_asyncio
nest_asyncio.apply()

# 3) On top of that index build query engine for retrieving the context.
query_engine = index.as_query_engine(llm = router_llm)

# 4) Take user query and generate an answer
user_query = "Tell me about attention block in LLMs briefly"
response = query_engine.query(user_query)
print(response)

2025-12-12 12:32:27,741 - INFO - AFC is enabled with max remote calls: 10.
2025-12-12 12:32:29,562 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"


The Transformer model utilizes multi-head attention in three distinct ways. In "encoder-decoder attention" layers, queries originate from the preceding decoder layer, while keys and values are sourced from the encoder's output. This enables every position in the decoder to access all positions within the input sequence, mirroring conventional encoder-decoder attention mechanisms in sequence-to-sequence models. The encoder itself incorporates self-attention layers, where keys, values, and queries all stem from the output of the previous encoder layer. This allows each encoder position to attend to all positions in the prior encoder layer. Similarly, self-attention layers in the decoder permit each decoder position to attend to all preceding positions, including itself. To maintain the auto-regressive property, leftward information flow is prevented in the decoder by masking out illegal connections within the scaled dot-product attention mechanism.


In [5]:
retrieved_nodes = query_engine.retrieve(user_query)
print(retrieved_nodes[-1])

Node ID: e9a68726-6c64-4d82-a3ab-02b2efcbc1de
Text: MultiHead(Q,K,V ) = Concat(head1,..., headh)WO where headi =
Attention(QWQ i ,KW K i ,VW V i ) Where the projections are parameter
matricesWQ i ∈Rdmodel×dk , WK i ∈Rdmodel×dk , WV i ∈Rdmodel×dv and WO
∈Rhdv×dmodel . In this work we employ h = 8 parallel attention layers,
or heads. For each of these we use dk = dv = dmodel/h= 64. Due to the
reduc...
Score:  0.850



### Add Qdrant database for more complicated RAG system

In [4]:
from qdrant_client import QdrantClient

# Setup qdrant client since it will be used as one of the parameters of the RAG System object
qdrant_client = QdrantClient(
    host = Config.QDRANT_URL,
    port = Config.QDRANT_PORT
)

In [5]:
from core.src.rag_system import RagSystem

# Instantiate an object of the RAG System
rag_test = RagSystem(
    qdrant_client = qdrant_client,
    router_llm = router_llm,
    embed_model = embed_model
)

In [6]:
import nest_asyncio
nest_asyncio.apply()

input_query = "Расскажи мне про квоты"
retrieved_nodes = rag_test.router_retriever.retrieve(input_query)

2025-12-12 15:53:35 | WARNING  | core.helpers.qdrant_setup:parse_to_query_result:94 - Payload text content is not a string (type: <class 'NoneType'>) for point 5d5d6408-4147-5d34-bc67-622bad306982. Converting to string.
2025-12-12 15:53:35 | WARNING  | core.helpers.qdrant_setup:parse_to_query_result:94 - Payload text content is not a string (type: <class 'NoneType'>) for point 83026f62-8110-5471-8539-55365cb62d0f. Converting to string.
2025-12-12 15:53:35 | WARNING  | core.helpers.qdrant_setup:parse_to_query_result:94 - Payload text content is not a string (type: <class 'NoneType'>) for point a3abcea1-749a-5288-88ab-267e42096e49. Converting to string.
2025-12-12 15:53:35 | WARNING  | core.helpers.qdrant_setup:parse_to_query_result:94 - Payload text content is not a string (type: <class 'NoneType'>) for point 73f44fd4-354b-5f4c-b1f5-4e2e56dfb558. Converting to string.
2025-12-12 15:53:35 | WARNING  | core.helpers.qdrant_setup:parse_to_query_result:94 - Payload text content is not a stri

In [ ]:
# Get and observe the output of the selector:
selector = rag_test.router_retriever._selector

from llama_index.core.schema import QueryBundle
query_bundle = QueryBundle(query_str="Расскажи мне про квоты")

tool_metadata_list = [t.metadata for t in rag_test._retriever_tools]

llm_output = selector._select(tool_metadata_list, query_bundle)

print(llm_output)  # This is the parsed LLM output

selections=[SingleSelection(index=1, reason='Этот набор данных предоставляет детализированную информацию о распределении государственных образовательных грантов, выделяемых в виде квот для определенных социальных категорий граждан. Данные сгруппированы по общим направлениям подготовки и показывают, какое количество грантов зарезервировано для каждой льготной категории в рамках государственного заказа на 2025-2026 учебный год.'), SingleSelection(index=11, reason="Коллекция охватывает темы образовательных грантов, квот приема для различных категорий граждан, правила приема иностранных граждан, а также включает таблицы и шкалы перевода баллов международных стандартизированных тестов (ЕНТ, SAT, ACT, IB, A Level, TOEFL, IELTS, KAZTEST) и требования к абитуриентам по областям образования, таким как 'Педагогические науки' и 'Здравоохранение'."), SingleSelection(index=12, reason='Коллекция содержит... қабылдау квоталары, шетел азаматтарына қойылатын талаптар және басқа да ұйымдастырушылық мәсе

In [ ]:
points, _ = qdrant_client.scroll(
    collection_name="kvotas_ru_new",
    limit=5,
)

points

[Record(id='33b35671-8749-55e8-9f86-20604662e9c7', payload={'group_keys': {'Общий код и классификация направлений подготовки': '6В05 Естественные науки, математика и статистика'}, 'records': [{'Общий код и классификация направлений подготовки': '6В05 Естественные науки, математика и статистика', 'Наименование квоты или дифференцированного гранта': 'Квота для детей-сирот и детей, оставшихся без попечения родителей, а также граждан Республики Казахстан из числа молодежи, потерявших или оставшихся без попечения родителей до совершеннолетия, - 1 процент', 'Количество квот': 92, 'Тип квоты': 'Квоты и дифференцированные гранты: государственный образовательный заказ на подготовку кадров с высшим образованием на 2025-2026 учебный год в разрезе групп образовательных программ'}, {'Общий код и классификация направлений подготовки': '6В05 Естественные науки, математика и статистика', 'Наименование квоты или дифференцированного гранта': 'Квота для лиц с инвалидностью первой или второй группы, лиц с

In [56]:
point = qdrant_client.retrieve(
    collection_name="kvotas_ru_new",
    ids=["33b35671-8749-55e8-9f86-20604662e9c7"],  # string or int
    with_vectors = False
)

dict(point[0])['payload']

{'group_keys': {'Общий код и классификация направлений подготовки': '6В05 Естественные науки, математика и статистика'},
 'records': [{'Общий код и классификация направлений подготовки': '6В05 Естественные науки, математика и статистика',
   'Наименование квоты или дифференцированного гранта': 'Квота для детей-сирот и детей, оставшихся без попечения родителей, а также граждан Республики Казахстан из числа молодежи, потерявших или оставшихся без попечения родителей до совершеннолетия, - 1 процент',
   'Количество квот': 92,
   'Тип квоты': 'Квоты и дифференцированные гранты: государственный образовательный заказ на подготовку кадров с высшим образованием на 2025-2026 учебный год в разрезе групп образовательных программ'},
  {'Общий код и классификация направлений подготовки': '6В05 Естественные науки, математика и статистика',
   'Наименование квоты или дифференцированного гранта': 'Квота для лиц с инвалидностью первой или второй группы, лиц с инвалидностью с детства, детей с инвалиднос

In [61]:
dict(retrieved_nodes[0].node)
type(retrieved_nodes[0])

llama_index.core.schema.NodeWithScore

In [7]:
print(rag_test._relevance_filter(retrieved_nodes, input_query))


        Question: Расскажи мне про квоты
        
        Retrieved context:
        None
None
None
None
None
None
None
None
None
None
При поступлении в организации высшего профессионального образования (ОВПО) устанавливается квота приема, размер которой утверждается приказом Министра науки и высшего образования. Данный приказ зарегистрирован в Реестре нормативных правовых актов. Изменения в пункт 8 вступят в силу после официального опубликования.
Документ описывает шкалу перевода баллов внешнего оценивания выпускников АОО 'НИШ' в баллы сертификата ЕНТ, включая максимальные баллы и соответствие оценок для различных предметов, таких как казахский и русский языки, история Казахстана и математика.
{
  "title": "-",
  "headers": [
    "№",
    "Предметы",
    "Для выпускников АОО \"НИШ\"",
    "Для выпускников АОО \"НИШ\"",
    "Для выпускников АОО \"НИШ\"",
    "Для выпускников АОО \"НИШ\"",
    "Для выпускников АОО \"НИШ\"",
    "Для выпускников АОО \"НИШ\"",
    "Для выпускников АОО \"